## keras로 모델을 만들기 위해 다양한 API를 제공한다.

1. sequential 
2. functional
3. subclassing

-------

각 사용에 대해 예시 코드로 파악해보자

In [1]:
# sequential 모델
# 가장 간단한 방법: 단순히 층을 쌓자

from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(64, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
])


Metal device set to: Apple M1


In [4]:
# sequential은 모델의 가중치를 층을 호출할 때 만들지 못한다. 입력 크기가 제공되지 않기 때문
# 따라서 build메서드를 호출하여 입력 크기를 지정하여 호출해야 한다.

model.build(input_shape = (None, 3))
model.weights

[<tf.Variable 'dense/kernel:0' shape=(3, 64) dtype=float32, numpy=
 array([[-0.06772433,  0.20004049,  0.20425594, -0.11676213, -0.17820448,
          0.1320768 , -0.00096968, -0.13764225, -0.06281976,  0.09221911,
         -0.0998107 ,  0.22431225,  0.23264647, -0.29569814, -0.20777142,
          0.20608604, -0.01511472, -0.13858004,  0.24877274, -0.12259023,
         -0.2170538 , -0.03970078,  0.12425283, -0.25118834,  0.23179954,
          0.2832911 , -0.16436271,  0.08530957, -0.27809122,  0.0267894 ,
         -0.28984272,  0.00082165, -0.24373794, -0.2871203 , -0.27497828,
         -0.02926886,  0.01187348, -0.20560089, -0.29400876,  0.13333651,
         -0.2876745 ,  0.02863193, -0.21602711,  0.15187329, -0.26810622,
         -0.159156  , -0.11384888,  0.07997161, -0.2788737 ,  0.11477596,
         -0.10879841, -0.10674217, -0.03367984, -0.2759558 ,  0.15279579,
         -0.24691412,  0.02053136,  0.15234709, -0.19214854, -0.06559554,
          0.05271527,  0.09685647, -0.0731978

In [5]:
# 모델 구조 출력 : summary() <- 모델의 build()메서드를 호출해야 만들어낼 수 있다.

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  256       
                                                                 
 dense_1 (Dense)             multiple                  650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


In [7]:
# build()메서드 호출 안하고 모델의 파라미터 크기 추정하는 법: input 클래스 사용

model = keras.Sequential()
#input클래스의 shape매개변수는 배치크기를 고려하지 않고, 샘플의 크기만 고려한다.
model.add(keras.Input(shape = (3,)))
model.add(layers.Dense(64, activation = 'relu'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                256       
                                                                 
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [8]:
# functional api
# 다중입력, 다중출력
# input 객체 -> 층만들어 input객체의 입력으로 호출 -> 최종 출력을 얻고 입력과 최종출력을 model클래스에 전달하여 모델 객체 생성

#다중입력, 다중출력 함수형 모델 예시
vocabulary_size = 10000
num_tags = 100
num_departments = 4

title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

features = layers.Concatenate()([title, text_body, tags])
features = layers.Dense(64, activation="relu")(features)

priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(num_departments, activation="softmax", name="department")(features)

model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 title (InputLayer)             [(None, 10000)]      0           []                               
                                                                                                  
 text_body (InputLayer)         [(None, 10000)]      0           []                               
                                                                                                  
 tags (InputLayer)              [(None, 100)]        0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 20100)        0           ['title[0][0]',                  
                                                                  'text_body[0][0]',          

### functional api로 모델흥 훈련할 때는 입력과 출력 데이터의 리스트를 전달해줄 것!

가령 위의 예시코드에서 title, text_body, tags의 입력 데이터가 있고 priority, department의 타겟 데이터가 있다고 가정하면  

훈련은   
**model.fit([title, text_body, tags], [priority, department])**  
로 인자를 전달!

**중요 포인트: 모델 클래스에 전달된 순서대로 리스트의 순서를 구성할 것!**

In [35]:
#함수형 모델을 쓰면 좋은점: 그래프를 그려볼 수 있어 모델 구조를 직관적으로 이해할 수 있다.

keras.utils.plot_model(model, "ticket_classifier.png", show_shapes = True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


오류가 났다... 
내용은 설치를 하면 already satisfied라고 하는데 해당 코드블럭을 실행하면 
"You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work."

해결해보자! -> 환경변수 설정 해봤는데 안됐다 ...

In [37]:
# subclassing model

class myticketmodel(keras.Model):
    def __init__(self, num_department):
        super().__init__()
        # 사용할 층을 정의
        self.concat_layer = layers.Concatenate()
        self.mixing_laye = layers.Dense(64, activation = 'relu')
        self.priority_score = layers.Dense(1, activation = 'sigmoid')
        self.department_classifier = layers.Dense(num_department, activation = 'softmax')
        
    def call(self, inputs):
        #정방형 패스
        title = inputs['title']
        text_body = inputs['text_body']
        tags = inputs['tags']
        
        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.prioirity_score(features)
        department = self.department_claasifier(features)
        return priority, department

model = myticketmodel(4)


In [ ]:
priority, department = model([title, text_body, tags (의ata)])

In [ ]:
#모델을 상속받으면 fit,eval, pred가능하다

model.compile(optimizer="rmsprop",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])
model.fit({"title": title_data,
           "text_body": text_body_data,
           "tags": tags_data},
          [priority_data, department_data],
          epochs=1)
model.evaluate({"title": title_data,
                "text_body": text_body_data,
                "tags": tags_data},
               [priority_data, department_data])
priority_preds, department_preds = model.predict({"title": title_data,
                                                  "text_body": text_body_data,
                                                  "tags": tags_data})

In [38]:
# 여러방식 혼합해서 사용하기 -> 3가지 방법이 상호 호환적이다.

#혼합 예시1: 함수형 모델에서 서브클래싱 층이나 모델 사용하기
class Classifier(keras.Model):

    def __init__(self, num_classes=2):
        super().__init__()
        if num_classes == 2:
            num_units = 1
            activation = "sigmoid"
        else:
            num_units = num_classes
            activation = "softmax"
        self.dense = layers.Dense(num_units, activation=activation)

    def call(self, inputs):
        return self.dense(inputs)

inputs = keras.Input(shape=(3,))

features = layers.Dense(64, activation="relu")(inputs)
outputs = Classifier(num_classes=10)(features)
model = keras.Model(inputs=inputs, outputs=outputs)

In [39]:
#혼합 예시2: 함수형 모델을 포함하는 subclassing 층이나 모델

inputs = keras.Input(shape=(64,))
outputs = layers.Dense(1, activation="sigmoid")(inputs)
binary_classifier = keras.Model(inputs=inputs, outputs=outputs)

class MyModel(keras.Model):

    def __init__(self, num_classes=2):
        super().__init__()
        self.dense = layers.Dense(64, activation="relu")
        self.classifier = binary_classifier

    def call(self, inputs):
        features = self.dense(inputs)
        return self.classifier(features)

model = MyModel()

## 내장된 훈련 루프 & 평가 루프

fit() / compile() / evaluate() / predict() 워크플로 사용

-> 이를 커스터 마이징 할 수 있다.

In [45]:
# 커스터 마이징 1. 사용자 정의 지표 만들기
# 즉, metrics을 우리가 새로 정의해서 적용해보자!

#metric 클래스 상속하여 사용자 정의 지표 구현 예시

import tensorflow as tf

class RMSE(keras.metrics.Metric):
    def __init__(self, name = 'rmse', **kwargs):
        super().__init__(name, **kwargs)
        self.mse_sum = self.add_weight(name = 'mse_sum', initializer = 'zeros')
        self.total_samples = self.add_weight(name = 'total_sample', initializer = 'zeros', dtype = "int32")
    
    
    # update_state메서드로 상태 업데이트 로직을 구현.
    def update_state(self, y_true, y_pred, sample_weight= None):
        y_true = tf.one_hot(y_true, depth = tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)
    
    #현재 지표 값을 반환.
    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    #객체를 다시 새로 생성하지 않으면서 상태를 초기화
    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

In [46]:
# 사용자 정의 지표 사용하기
from tensorflow.keras.datasets import mnist

def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy", RMSE()])

model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))

test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'


1562/1563 [============================>.] - ETA: 0s - loss: 0.2863 - accuracy: 0.9150 - rmse: 7.1964

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x16x1x512xi1>'


1563/1563 [==============================] - ETA: 0s - loss: 0.2862 - accuracy: 0.9150 - rmse: 7.1958

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x16x1x512xi1>'


1563/1563 [==============================] - 21s 12ms/step - loss: 0.2862 - accuracy: 0.9150 - rmse: 7.1958 - val_loss: 0.1519 - val_accuracy: 0.9567 - val_rmse: 7.3626
Epoch 2/3
1563/1563 [==============================] - 16s 10ms/step - loss: 0.1553 - accuracy: 0.9570 - rmse: 7.3629 - val_loss: 0.1241 - val_accuracy: 0.9666 - val_rmse: 7.4079
Epoch 3/3
313/313 [==============================] - 2s 6ms/step - loss: 0.1225 - accuracy: 0.9683 - rmse: 7.4336


In [47]:
# modelcheckpoint + earlystopping callback 사용 => 최적의 에포크 횟수로 학습

#fit메서드의 callback 매개변수 사용.
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=2,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True,
    )
]
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=10,
          callbacks=callbacks_list,
          validation_data=(val_images, val_labels))

Epoch 1/10


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'


1562/1563 [============================>.] - ETA: 0s - loss: 0.2804 - accuracy: 0.9179

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x16x1x512xi1>'


1563/1563 [==============================] - ETA: 0s - loss: 0.2803 - accuracy: 0.9179

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x16x1x512xi1>'


1563/1563 [==============================] - 19s 11ms/step - loss: 0.2803 - accuracy: 0.9179 - val_loss: 0.1658 - val_accuracy: 0.9531
Epoch 2/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.1550 - accuracy: 0.9564 - val_loss: 0.1359 - val_accuracy: 0.9633
Epoch 3/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.1240 - accuracy: 0.9680 - val_loss: 0.1250 - val_accuracy: 0.9698
Epoch 4/10
1563/1563 [==============================] - 14s 9ms/step - loss: 0.1105 - accuracy: 0.9707 - val_loss: 0.1183 - val_accuracy: 0.9709
Epoch 5/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.1030 - accuracy: 0.9742 - val_loss: 0.1174 - val_accuracy: 0.9731
Epoch 6/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.0956 - accuracy: 0.9773 - val_loss: 0.1070 - val_accuracy: 0.9754
Epoch 7/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.0890 - accuracy: 0.9784 - val_loss: 0.1177 - val_accuracy: 0.

In [ ]:
# 저장한 모델 호출하기

#modelcheckpoint의 매개변수로 저장한 파일 이름으로 불러온다.
best_model = keras.models.load_model('checkpoint_path.keras')

In [ ]:
# 사용자 정의 콜백 만들기

from matplotlib import pyplot as plt

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs):
        self.per_batch_losses = []

    def on_batch_end(self, batch, logs):
        self.per_batch_losses.append(logs.get("loss"))

    def on_epoch_end(self, epoch, logs):
        plt.clf()
        plt.plot(range(len(self.per_batch_losses)), self.per_batch_losses,
                 label="Training loss for each batch")
        plt.xlabel(f"Batch (epoch {epoch})")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(f"plot_at_epoch_{epoch}")
        self.per_batch_losses = []

## 텐서보드 사용 -> 훈련하는 동안 모델에서 일어나는 것들을 모니터링해보자

fit()메서드에 keras.callbacks.TensorBoard 콜백을 사용.

In [48]:
# 예시

model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

tensorboard = keras.callbacks.TensorBoard(
    log_dir="./tb_logs",
)

model.fit(train_images, train_labels,
          epochs=10,
          validation_data=(val_images, val_labels),
          callbacks=[tensorboard])

Epoch 1/10


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'


1558/1563 [============================>.] - ETA: 0s - loss: 0.2865 - accuracy: 0.9160

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x16x1x512xi1>'


1563/1563 [==============================] - ETA: 0s - loss: 0.2861 - accuracy: 0.9161

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x16x1x512xi1>'


1563/1563 [==============================] - 19s 11ms/step - loss: 0.2861 - accuracy: 0.9161 - val_loss: 0.1538 - val_accuracy: 0.9559
Epoch 2/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.1578 - accuracy: 0.9554 - val_loss: 0.1243 - val_accuracy: 0.9648
Epoch 3/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.1278 - accuracy: 0.9657 - val_loss: 0.1333 - val_accuracy: 0.9677
Epoch 4/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.1113 - accuracy: 0.9709 - val_loss: 0.1241 - val_accuracy: 0.9706
Epoch 5/10
1563/1563 [==============================] - 15s 10ms/step - loss: 0.1047 - accuracy: 0.9741 - val_loss: 0.1136 - val_accuracy: 0.9729
Epoch 6/10
1563/1563 [==============================] - 15s 9ms/step - loss: 0.0946 - accuracy: 0.9758 - val_loss: 0.1206 - val_accuracy: 0.9751
Epoch 7/10
1563/1563 [==============================] - 15s 10ms/step - loss: 0.0916 - accuracy: 0.9784 - val_loss: 0.1180 - val_accuracy: 

In [ ]:
# 텐서보드 실행시켜서 나오는 url클릭해서 모니터링 !

!tensorboard --logdir ./tb_logs

## 사용자 정의 훈련 + 평가 루프 만들기

model.fit()메서드를 활용하면 사용자 정의 손실, 지표, 콜백을 지정하여 보다 유연하게 
모델 학습을 할 수 있었다.

다만, 이는 지도 학습에만 초점이 맞춰져있기 때문에, 생성학습, 자기지도학습, 강화학습 등에서 모델을   
학습하고 평가하기 위해서는 저수준의 유연하게 동작하도록 기능을 추가하는 작업이 필요하다.  
이 때 일반적으로 포함되어야 하는 내용은 다음과 같다.   

#### 1. gradientTape에서 정방향 패스 -> 손실 값 측정
#### 2. 모델 가중치에 대한 손실의 gradient 계산
#### 3. 현재 배치 데이터에 대한 손실 값을 낮추는 방향으로 모델 가중치를 업데이트


In [2]:
#저수준 측정 지표 예시
from tensorflow import keras

metric = keras.metrics.SparseCategoricalAccuracy()
targets = [0, 1, 2]
predictions = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
metric.update_state(targets, predictions)
current_result = metric.result()
print(f"결과: {current_result:.2f}")

values = [0, 1, 2, 3, 4]
mean_tracker = keras.metrics.Mean()
for value in values:
    mean_tracker.update_state(value)
print(f"평균 지표: {mean_tracker.result():.2f}")

Metal device set to: Apple M1
결과: 1.00
평균 지표: 2.00


In [5]:
#훈련 루프 구성하기
from tensorflow.keras.datasets import mnist

#모델 구성
def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = keras.layers.Dense(512, activation="relu")(inputs)
    features = keras.layers.Dropout(0.5)(features)
    outputs = keras.layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model


#모델 정의
model = get_mnist_model()

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.legacy.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = keras.metrics.Mean()

def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["loss"] = loss_tracking_metric.result()
    return logs

def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()


In [7]:
import tensorflow as tf

#데이터 준비
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

#dataset tensor
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
training_dataset = training_dataset.batch(32)

# 훈련 루프
epochs = 3
for epoch in range(epochs):
    reset_metrics()
    for inputs_batch, targets_batch in training_dataset:
        logs = train_step(inputs_batch, targets_batch)
    print(f"{epoch}번째 에포크 결과")
    for key, value in logs.items():
        print(f"...{key}: {value:.4f}")

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerforman

0번째 에포크 결과
...sparse_categorical_accuracy: 0.9179
...loss: 0.2775
1번째 에포크 결과
...sparse_categorical_accuracy: 0.9654
...loss: 0.1246
2번째 에포크 결과
...sparse_categorical_accuracy: 0.9802
...loss: 0.0788


In [8]:
#평가루프 구성
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return logs

#평가 루프
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print("평가 결과:")
for key, value in logs.items():
    print(f"...{key}: {value:.4f}")

평가 결과:
...val_sparse_categorical_accuracy: 0.9587
...val_loss: 0.1797


In [9]:
#전역 최적화를 위한 tf.function 데코레이터 사용

@tf.function
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print("평가 결과:")
for key, value in logs.items():
    print(f"...{key}: {value:.4f}")

2023-10-26 12:28:22.704025: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


평가 결과:
...val_sparse_categorical_accuracy: 0.9587
...val_loss: 0.1797


In [10]:
# fit()이 사용할 사용자 정의 훈련 스텝 구현하기

loss_fn = keras.losses.SparseCategoricalCrossentropy()
loss_tracker = keras.metrics.Mean(name="loss")

class CustomModel(keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = loss_fn(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))

        loss_tracker.update_state(loss)
        return {"loss": loss_tracker.result()}

    @property
    def metrics(self):
        return [loss_tracker]

In [13]:
inputs = keras.Input(shape=(28 * 28,))
features = keras.layers.Dense(512, activation="relu")(inputs)
features = keras.layers.Dropout(0.5)(features)
outputs = keras.layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(optimizer=keras.optimizers.legacy.RMSprop())
model.fit(train_images, train_labels, epochs=3)

Epoch 1/3


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'


1558/1563 [============================>.] - ETA: 0s - loss: 0.2856

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x16x1x512xi1>'


1563/1563 [==============================] - 14s 7ms/step - loss: 0.2851
Epoch 2/3
   1/1563 [..............................] - ETA: 20s - loss: 0.0847

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x16x1x512xi1>'


1563/1563 [==============================] - 11s 7ms/step - loss: 0.1572
Epoch 3/3
1563/1563 [==============================] - 11s 7ms/step - loss: 0.1261


In [15]:
class CustomModel(keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            #손실계산
            loss = self.compiled_loss(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        #모델 지표 업데이트
        self.compiled_metrics.update_state(targets, predictions)
        return {m.name: m.result() for m in self.metrics}

inputs = keras.Input(shape=(28 * 28,))
features = keras.layers.Dense(512, activation="relu")(inputs)
features = keras.layers.Dropout(0.5)(features)
outputs = keras.layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(optimizer=keras.optimizers.legacy.RMSprop(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

model.fit(train_images, train_labels, epochs=3)

Epoch 1/3


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x512xi1>'


1561/1563 [============================>.] - ETA: 0s - loss: 0.2853 - sparse_categorical_accuracy: 0.9160

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x16x1x512xi1>'


1563/1563 [==============================] - 16s 9ms/step - loss: 0.2853 - sparse_categorical_accuracy: 0.9160
Epoch 2/3
   1/1563 [..............................] - ETA: 18s - loss: 0.0507 - sparse_categorical_accuracy: 0.9688

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x16x1x512xi1>'


1563/1563 [==============================] - 14s 9ms/step - loss: 0.1545 - sparse_categorical_accuracy: 0.9563
Epoch 3/3
1563/1563 [==============================] - 14s 9ms/step - loss: 0.1277 - sparse_categorical_accuracy: 0.9654
